# 1.1.2.3: Transformaciones Lineales, Base y Dimensión

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Definir** formalmente una **transformación lineal** y verificar sus propiedades (preservación de la suma y del escalado).
- **Comprender** los conceptos de **Base** y **Dimensión** como los "ladrillos" y el "tamaño" de un espacio vectorial.
- **Encontrar la matriz estándar** para una transformación lineal, derivando por qué este método funciona.
- **Definir, calcular e interpretar** el **Kernel (espacio nulo)** y la **Imagen (espacio columna)** de una transformación.
- **Conectar** estos conceptos con aplicaciones como el **Análisis de Componentes Principales (PCA)** y la detección de redundancia de datos.

In [ ]:
# --- Celda de Configuración (Oculta) ---
%display latex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import null_space # Herramienta clave para el Kernel

def plot_linear_transformation(matrix, ax=None, title=None):
    standalone = ax is None
    if standalone:
        fig, ax = plt.subplots(figsize=(7, 7))
    
    limit = 4
    ax.set_xlim(-limit, limit); ax.set_ylim(-limit, limit)
    ax.set_aspect('equal'); ax.grid(True, linestyle='--')
    ax.axhline(0, color='black', lw=0.5); ax.axvline(0, color='black', lw=0.5)
    
    # Transforma la cuadrícula
    for i in range(-limit, limit + 1):
        p1 = matrix @ np.array([i, -limit]); p2 = matrix @ np.array([i, limit])
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], color='gray', lw=0.5, zorder=0)
        p1 = matrix @ np.array([-limit, i]); p2 = matrix @ np.array([limit, i])
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], color='gray', lw=0.5, zorder=0)
    
    # Transforma los vectores base
    i_hat_T = matrix @ np.array([1, 0]); j_hat_T = matrix @ np.array([0, 1])
    ax.quiver(0, 0, i_hat_T[0], i_hat_T[1], angles='xy', scale_units='xy', scale=1, color='#0072B2', zorder=2, label='T(î)')
    ax.quiver(0, 0, j_hat_T[0], j_hat_T[1], angles='xy', scale_units='xy', scale=1, color='#E69F00', zorder=2, label='T(ĵ)')
    
    ax.legend()
    if title:
        ax.set_title(title)
    elif standalone:
        ax.set_title(f"Transformación por la Matriz A = {np.round(matrix, 2).tolist()}")
        
    if standalone:
        plt.show()

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

Para un tema tan conceptual como las transformaciones lineales, la visualización es clave. Usaremos nuestros generadores para crear tanto datos estructurados (formas geométricas) como matrices con propiedades específicas (singulares, proyecciones) para hacer tangibles las ideas abstractas.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_geometric_shapes import create_geometric_shapes
from src.data_generation.create_special_matrices import create_special_matrices

# Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
rng = np.random.default_rng(seed=42)

# === Generación de Datasets y Matrices para este Notebook ===

# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor (Datos a transformar)
# El dataset de estudiantes nos sirve como un ejemplo de 'nube de puntos' del mundo real.
# Veremos cómo una transformación lineal afecta a la distribución de estos datos.
datos_estudiantes = create_student_performance_data(rng, simplified=True, n_samples=100)

# 💡 CONTEXTO PEDAGÓGICO: Lienzo para Transformaciones
# Una forma geométrica bien definida es el mejor 'lienzo' para pintar nuestras transformaciones
# y entender visualmente qué es una rotación, proyección o cizalla.
datos_lienzo = create_geometric_shapes(rng, shape_type='circle', n_samples=100, noise_level=0)

# 💡 CONTEXTO PEDAGÓGICO: Matrices para Kernel/Imagen
# Para entender el Kernel y la Imagen, necesitamos matrices que no tengan rango completo.
# Una matriz de proyección o una singular son ejemplos perfectos.
matriz_proyeccion = np.array([[1, 0], [0, 0]])
matriz_singular = create_special_matrices(rng, matrix_type='singular', size=(3, 3))

print("Datasets y matrices generados y listos para usar.")

## 1. La Definición Formal de Transformación Lineal

Una **transformación lineal** es una función $T$ que mapea vectores de un espacio a otro (e.g., de $\mathbb{R}^n$ a $\mathbb{R}^m$) de una forma que "respeta" la estructura del espacio vectorial. Específicamente, cumple dos reglas de oro:

1.  **Preservación de la Suma:** $T(\vec{u} + \vec{v}) = T(\vec{u}) + T(\vec{v})$
2.  **Preservación de la Multiplicación por Escalar:** $T(c\vec{v}) = cT(\vec{v})$

**Consecuencias Geométricas:** Estas reglas implican que **el origen no se mueve** ($T(\vec{0}) = \vec{0}$), y **las líneas de la cuadrícula permanecen paralelas y equidistantes** tras la transformación. Cualquier transformación que curve las líneas o mueva el origen no es lineal.

### Ejemplo Demostrativo 1: Verificando la Linealidad (y la no linealidad)

In [ ]:
# DATOS
u = np.array([1, 2]); v = np.array([3, -1]); c = 2

# CASO 1: Transformación Lineal (Rotación 90°)
T_lineal = lambda x: np.array([-x[1], x[0]])
lado_izq_suma = T_lineal(u + v)
lado_der_suma = T_lineal(u) + T_lineal(v)
print("--- Transformación Lineal (Rotación) ---")
print(f"T(u+v) = {lado_izq_suma}, T(u)+T(v) = {lado_der_suma}")
print(f"¿Propiedad de suma se cumple? {np.allclose(lado_izq_suma, lado_der_suma)}")

# CASO 2: Transformación NO Lineal (Traslación)
T_no_lineal = lambda x: x + np.array([1, 1])
lado_izq_suma_nl = T_no_lineal(u + v)
lado_der_suma_nl = T_no_lineal(u) + T_no_lineal(v)
print("\n--- Transformación NO Lineal (Traslación) ---")
print(f"T(u+v) = {lado_izq_suma_nl}, T(u)+T(v) = {lado_der_suma_nl}")
print(f"¿Propiedad de suma se cumple? {np.allclose(lado_izq_suma_nl, lado_der_suma_nl)}")
print("La traslación no es lineal porque el origen se mueve y la suma no se preserva.")

## 2. El Rol de la Base: Los "Ladrillos" del Espacio

Una **Base** para un espacio vectorial es un conjunto de vectores que cumple dos condiciones:
1.  Es **linealmente independiente** (no hay información redundante).
2.  **Genera (spans)** todo el espacio (puedes construir cualquier vector del espacio con ellos).

Una base es el conjunto de "ladrillos" mínimo y no redundante necesario para construir un espacio. La **dimensión** de un espacio vectorial es simplemente el **número de vectores** en cualquiera de sus bases. Para $\mathbb{R}^n$, la dimensión es $n$.

> **Aplicación a Data Science (PCA):** El **Análisis de Componentes Principales (PCA)** es un algoritmo para encontrar una **nueva base** más eficiente para describir los datos. Los componentes principales (vectores propios de la matriz de covarianza) son los vectores de esta nueva base, ordenados por la cantidad de varianza que explican. Cambiar de base es cambiar nuestro "punto de vista" para ver los datos de una forma más informativa.

### Ejemplo Demostrativo 2: Distintas Bases para el mismo Espacio

In [ ]:
# La base canónica es nuestro sistema de coordenadas estándar (ejes X e Y)
e1 = np.array([1, 0])
e2 = np.array([0, 1])

# Esta es otra base válida para R^2. Está rotada y no es ortogonal.
b1 = np.array([2, 1])
b2 = np.array([-1, 1])

# Un vector v
v = np.array([3, 5])

# Encontramos las coordenadas de v en la nueva base resolviendo B*c = v
B = np.column_stack([b1, b2])
coords_en_B = np.linalg.solve(B, v)

print(f"El vector v = {v} en la base canónica.")
print(f"Para construirlo en la nueva base necesitamos {coords_en_B[0]:.2f} del vector b1 y {coords_en_B[1]:.2f} del vector b2.")
print(f"Verificación: {coords_en_B[0]:.2f} * {b1} + {coords_en_B[1]:.2f} * {b2} = {coords_en_B[0]*b1 + coords_en_B[1]*b2}")


## 3. La Matriz Estándar de una Transformación Lineal

**Teorema Fundamental:** Toda transformación lineal $T: \mathbb{R}^n \to \mathbb{R}^m$ puede ser representada por la multiplicación por una única matriz $A$ de dimensión $m \times n$, tal que $T(\vec{x}) = A\vec{x}$.

Gracias a la linealidad, solo necesitamos saber a dónde van a parar los vectores de la base canónica ($\{\vec{e_1}, \vec{e_2}, \dots, \vec{e_n}\}$) para conocer toda la transformación.

**El "Truco" para Encontrar la Matriz A:** Las columnas de la matriz $A$ son, simple y elegantemente, las transformaciones de los vectores de la base canónica.
$$ A = [ \ T(\vec{e_1}) \ | \ T(\vec{e_2}) \ | \ \dots \ | \ T(\vec{e_n}) \ ] $$

### Ejemplo Demostrativo 3: Construyendo una Matriz de Transformación
**Problema:** Encontrar la matriz 2x2 para la transformación $T$ que rota 90° en sentido antihorario y luego aplica una cizalla (shear) horizontal (sumando la coordenada y a la x).

In [ ]:
# 1. Definimos los vectores de la base canónica
e1 = np.array([1, 0])
e2 = np.array([0, 1])

# 2. Aplicamos la transformación a e1
# Rotar [1, 0] -> [0, 1]. Cizallar [0, 1] -> [0+1, 1] = [1, 1]
T_e1 = np.array([1, 1])

# 3. Aplicamos la transformación a e2
# Rotar [0, 1] -> [-1, 0]. Cizallar [-1, 0] -> [-1+0, 0] = [-1, 0]
T_e2 = np.array([-1, 0])

# 4. Ensamblamos la matriz con los resultados como columnas
A = np.column_stack([T_e1, T_e2])

print(f"La primera columna de A es T(e1) = {T_e1}")
print(f"La segunda columna de A es T(e2) = {T_e2}")
print(f"\nMatriz de transformación estándar A:\n{A}")

# 5. Verificación visual
plot_linear_transformation(A, title='Transformación Compuesta: Rotación + Cizalla')

## 4. Kernel e Imagen: Los Subespacios Fundamentales

Toda transformación lineal tiene dos subespacios asociados que la describen completamente:

- **Imagen (o Espacio Columna):** Es el **rango** de la transformación; el conjunto de todos los posibles vectores de salida. Corresponde al **span de las columnas** de la matriz $A$. Su dimensión es el **rango** de la matriz.

- **Kernel (o Espacio Nulo):** El conjunto de todos los vectores de entrada $\vec{v}$ que son "aplastados" al origen por la transformación ($T(\vec{v}) = A\vec{v} = \vec{0}$). Si el Kernel contiene más que el vector cero, la transformación está perdiendo dimensiones.

> **Aplicación a Data Science:**
> - La **Imagen** (espacio columna) de una matriz de datos es el universo de todas las posibles predicciones que un modelo lineal puede generar.
> - Un **Kernel** no trivial ($\{\vec{0}\}$) implica que la matriz es singular, sus columnas son linealmente dependientes y existe **multicolinealidad**.

### Ejemplo Demostrativo 4: Analizando el Kernel y la Imagen de una Proyección

In [ ]:
# 1. DATOS: Una matriz de proyección 3D -> 2D
A = np.array([[1, 0, 2], [0, 1, -1], [0, 0, 0]]) # Proyecta sobre el plano x-y (de forma no ortogonal)

# 2. IMAGEN (Espacio Columna)
# La imagen es el span de las columnas. Las dos primeras son independientes.
# La tercera es una combinación de las dos primeras. Por tanto, la imagen es un plano 2D en R^3.
rango = np.linalg.matrix_rank(A)
print(f"--- ANÁLISIS DE LA IMAGEN ---")
print(f"El rango de A es {rango}, por lo que la dimensión de la Imagen es {rango}.")
print("Interpretación: Aunque los vectores de entrada están en 3D, todas las salidas viven en un plano 2D.")

# 3. KERNEL (Espacio Nulo)
# Usamos scipy.linalg.null_space para encontrar una base para el kernel.
base_kernel = null_space(A)
print(f"\n--- ANÁLISIS DEL KERNEL ---")
print(f"Una base para el Kernel de A es:\n{np.round(base_kernel, 2)}")
print(f"La dimensión del Kernel (nulidad) es {base_kernel.shape[1]}.")

# 4. TEOREMA RANGO-NULIDAD
nulidad = base_kernel.shape[1]
num_columnas = A.shape[1]
print(f"\nVerificación del Teorema Rango-Nulidad: Rango({rango}) + Nulidad({nulidad}) = {rango + nulidad} (Número de columnas)")

# Verificamos que un vector en el kernel es mapeado a cero
v_en_kernel = base_kernel[:, 0]
resultado = A @ v_en_kernel
print(f"\nAplicando A a un vector del Kernel: A @ v_k = {np.round(resultado, 5)}")

---
## 4. Ejercicios Guiados con Scaffolding (8+)
Rellena las partes marcadas con `# COMPLETAR` para afianzar tu comprensión.

### === EJERCICIO GUIADO 1: Verificar Propiedades de Linealidad ===

In [ ]:
# DATOS: Una matriz de transformación y dos vectores.
T = np.array([[2, -1], [1, 1]])
u = np.array([2, 3])
v = np.array([-1, 4])

# TODO 1: Verifica la propiedad de la suma T(u+v) = T(u) + T(v)
lado_izquierdo = # COMPLETAR. Aplica T a la suma de u y v.
lado_derecho = # COMPLETAR. Suma los resultados de aplicar T a u y v por separado.

# VERIFICACIÓN AUTOMÁTICA
assert np.allclose(lado_izquierdo, lado_derecho), "La propiedad de la suma no se cumple."
print(f"✅ Propiedad de la suma verificada: {lado_izquierdo} == {lado_derecho}")

### === EJERCICIO GUIADO 2: Encontrar la Matriz Estándar (Simple) ===

In [ ]:
# DESCRIPCIÓN: Una transformación T: R^2 -> R^2 que refleja los puntos sobre el eje X.

# TODO 1: Encuentra a dónde va a parar el primer vector base e1 = [1, 0].
T_e1 = # COMPLETAR

# TODO 2: Encuentra a dónde va a parar el segundo vector base e2 = [0, 1].
T_e2 = # COMPLETAR

# TODO 3: Ensambla la matriz estándar A usando T_e1 y T_e2 como columnas.
A = # COMPLETAR

# VERIFICACIÓN AUTOMÁTICA
A_esperada = np.array([[1, 0], [0, -1]])
assert np.allclose(A, A_esperada), "La matriz estándar es incorrecta."
print(f"✅ Matriz de reflexión sobre el eje X encontrada correctamente:\n{A}")

### === EJERCICIO GUIADO 3: Encontrar Matriz Estándar (R^2 -> R^3) ===

In [ ]:
# DESCRIPCIÓN: Una transformación T: R^2 -> R^3 que mapea (x, y) a (x, y, x+y).

# TODO 1: Aplica la transformación a e1 = [1, 0].
T_e1 = # COMPLETAR

# TODO 2: Aplica la transformación a e2 = [0, 1].
T_e2 = # COMPLETAR

# TODO 3: Ensambla la matriz A. ¿Qué forma (dimensiones) debería tener?
A = # COMPLETAR

# VERIFICACIÓN
assert A.shape == (3, 2), "La forma de la matriz es incorrecta para un mapeo R^2 -> R^3."
assert np.allclose(A @ np.array([2, 3]), np.array([2, 3, 5])), "La matriz no transforma correctamente un vector de prueba."
print(f"✅ Matriz estándar R^2 -> R^3 encontrada:\n{A}")

### === EJERCICIO GUIADO 4: Comprobar si un conjunto es una Base ===

In [ ]:
# DATOS: Un conjunto de 3 vectores en R^3.
v1 = np.array([1, 2, 3]); v2 = np.array([0, 1, 2]); v3 = np.array([-1, 0, 3])

# Para ser una base de R^3, un conjunto debe tener 3 vectores linealmente independientes.

# TODO 1: Crea una matriz B con estos vectores como columnas.
B = # COMPLETAR

# TODO 2: Calcula el rango de la matriz B.
rango_B = # COMPLETAR

# TODO 3: Escribe una condición booleana para determinar si es una base.
# PISTA: ¿Cómo se compara el rango con el número de vectores?
es_una_base = # COMPLETAR

assert es_una_base, "Estos vectores deberían formar una base."
print(f"✅ El conjunto es una base para R^3 porque tiene 3 vectores y su rango es {rango_B}.")

### === EJERCICIO GUIADO 5: Encontrar Coordenadas en una Nueva Base ===

In [ ]:
# DATOS: La base B del ejercicio anterior y un vector 'v'.
B = np.column_stack([[1, 2, 3], [0, 1, 2], [-1, 0, 3]])
v = np.array([8, 7, 6])

# Queremos encontrar los coeficientes 'c' tal que B @ c = v.

# TODO: Resuelve el sistema de ecuaciones para encontrar el vector de coordenadas 'c'.
# PISTA: Usa np.linalg.solve().
c = # COMPLETAR

# VERIFICACIÓN
assert np.allclose(B @ c, v), "La combinación lineal de las coordenadas no reconstruye el vector original."
print(f"El vector v = {v} se representa como {np.round(c, 2)} en la base B.")

### === EJERCICIO GUIADO 6: Encontrar el Kernel (Espacio Nulo) ===

In [ ]:
# DATOS: Una matriz 3x3 con columnas linealmente dependientes.
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# TODO: Usa la función importada `null_space` para encontrar una base para el kernel de A.
base_del_kernel = # COMPLETAR

# VERIFICACIÓN
# 1. El resultado no debe estar vacío.
assert base_del_kernel.shape[1] > 0, "Esta matriz debería tener un kernel no trivial."
# 2. Al multiplicar la matriz por un vector del kernel, el resultado debe ser (casi) cero.
vector_del_kernel = base_del_kernel[:, 0] # Tomamos el primer (y único) vector de la base
resultado = A @ vector_del_kernel
assert np.allclose(resultado, np.zeros(3)), "A @ v_k no es el vector cero."
print("✅ Base para el Kernel encontrada correctamente.")
print(f"Un vector que es 'aplastado' al origen por A es {np.round(vector_del_kernel, 2)}.")

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento.

### Parte A: Verificación de Linealidad (Conceptuales)

**A1 (🟢 Fácil):** ¿Es la transformación $T(\vec{v}) = 5\vec{v}$ lineal? Verifica las dos propiedades con un ejemplo numérico.

**A2 (🟢 Fácil):** ¿Es la transformación $T(x, y) = (x+y, x-y)$ lineal? Verifica las dos propiedades.

**A3 (🟡 Medio):** ¿Es la transformación $T(x, y) = (xy, y)$ lineal? Justifica tu respuesta mostrando cuál de las dos propiedades falla.

**A4 (🟡 Medio):** ¿Es la transformación "calcular la norma L2", $T(\vec{v}) = ||\vec{v}||_2$, una transformación lineal de $\mathbb{R}^2 \to \mathbb{R}$? Justifica.

### Parte B: Encontrar la Matriz Estándar

**B1 (🟢 Fácil):** Encuentra la matriz estándar para la transformación $T(x, y) = (y, x)$ (una reflexión sobre la línea y=x).

**B2 (🟢 Fácil):** Encuentra la matriz estándar para la transformación de proyección sobre el eje y, $T(x,y) = (0, y)$.

**B3 (🟡 Medio):** Encuentra la matriz estándar para la transformación $T: \mathbb{R}^2 \to \mathbb{R}^2$ que rota los vectores 180 grados.

**B4 (🟡 Medio):** Encuentra la matriz estándar para la transformación $T: \mathbb{R}^3 \to \mathbb{R}^2$ definida por $T(x, y, z) = (x+z, y-z)$.

**B5 (🔴 Reto):** Encuentra la matriz estándar para la transformación $T: \mathbb{R}^2 \to \mathbb{R}^2$ que primero aplica una cizalla vertical ($T(x,y)=(x, y+0.5x)$) y luego refleja sobre el eje y.

### Parte C: Bases y Dimensión

**C1 (🟢 Fácil):** ¿El conjunto `{[1, 2], [2, 4]}` forma una base para ℝ²? Justifica.

**C2 (🟢 Fácil):** ¿Cuántos vectores necesitas para formar una base de $\mathbb{R}^4$?

**C3 (🟡 Medio):** Verifica si el conjunto de vectores `{[1,0,1], [1,1,0], [0,1,1]}` forma una base para $\mathbb{R}^3$.

**C4 (🔴 Reto):** Considera la base $B = \{ [1,1], [-1,1] \}$. Encuentra las coordenadas del vector $\vec{v}=[5, 3]$ en esta base $B$.

### Parte D: Kernel e Imagen

**D1 (🟢 Fácil):** Describe geométricamente la Imagen y el Kernel de una rotación de 45 grados en $\mathbb{R}^2$.

**D2 (🟡 Medio):** Encuentra una base para la Imagen de la matriz $A = [[1, 2, 3], [4, 5, 6]]$. (Pista: el espacio columna).

**D3 (🟡 Medio):** Encuentra una base para el Kernel de la matriz del ejercicio anterior, $A = [[1, 2, 3], [4, 5, 6]]$. Verifica el teorema Rango-Nulidad.

**D4 (🔴 Reto):** Construye una matriz 3x3 (que no sea la matriz cero) cuya Imagen sea una línea y su Kernel sea un plano. (Pista: piensa en una matriz de rango 1).

---

## ✅ Mini-Quiz de Autoevaluación

1. ¿Cuáles son las dos condiciones que debe cumplir una función para ser una transformación lineal?
2. Si una transformación lineal $T: \mathbb{R}^2 \to \mathbb{R}^2$ transforma $\vec{e_1}$ en $[5,0]$ y $\vec{e_2}$ en $[1,2]$, ¿cuál es su matriz estándar?
3. Si el kernel de una transformación $T: \mathbb{R}^5 \to \mathbb{R}^5$ tiene dimensión 2, ¿cuál es la dimensión de su imagen?
4. Verdadero o Falso: Cualquier conjunto de 3 vectores en $\mathbb{R}^3$ forma una base para $\mathbb{R}^3$.

## 🚀 Próximos Pasos

Hemos sentado las bases teóricas de las transformaciones. Ahora estamos listos para explorar propiedades más profundas y herramientas para analizarlas.

- En **`1.1.3.1_Producto_Punto`**, exploraremos una operación que nos permite medir ángulos y proyecciones, propiedades geométricas clave que son alteradas por estas transformaciones y que son fundamentales para algoritmos como SVM y OLS.